In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# 第一种方式，直接图像分块：
def image2emb_naive(image,patch_size,weight):
    #unfold 函数的返回值[N,C*W*H,L]
    patch = F.unfold(image,kernel_size=patch_size,stride=patch_size).transpose(-1,-2)
    #图像拆分成patch之后，和权重矩阵相乘，就得到了每个batch Embedding之后的维度：
    patch_embedding = torch.matmul(patch,weight)

    return patch_embedding

bs,ic,image_h,image_w = 1,3,8,8
patch_size = 4
model_dim = 8 #目标要输出的通道数目；
patch_depth = patch_size*patch_size*ic
image = torch.randn(bs,ic,image_h,image_w)
weight = torch.randn(patch_depth,model_dim)

patch_embedding_navie = image2emb_naive(image,patch_size,weight)
print(patch_embedding_navie.shape,patch_embedding_navie)

torch.Size([1, 4, 8]) tensor([[[ 14.4183,   4.0192,   4.9964,  -9.4050,   7.4996,  -1.2957,  10.1465,
           -7.6784],
         [-11.7635,   7.4509,   0.2277, -16.4446,  -4.2662,   0.7789,   7.5461,
           -3.1969],
         [ -6.7492,   4.5716,   7.7062,  -4.8321,   3.8501,  -6.5702, -14.2348,
            5.2437],
         [  0.8196,   9.2905,  -2.3660,   4.6366,  -5.8006,   4.6970,   0.5125,
           -4.0809]]])


In [4]:

# 第二种方式，卷积的方式：
def image2emb_conv(image,kernel,stride):
    conv_output = F.conv2d(image,kernel,stride=stride)
    bs,oc,ow,oh = conv_output.shape

    # 还是那个事，transfomer的输入形状是 NLC，L在图像中就是 W*H ，也就是序列长度；
    patch_embedding = conv_output.reshape((bs,oc,ow*oh)).transpose(-1,-2)
    return patch_embedding
bs,ic,image_h,image_w = 1,3,8,8
patch_size = 4
model_dim = 8 #目标要输出的通道数目；
patch_depth = patch_size*patch_size*ic
# image = torch.randn(bs,ic,image_h,image_w)
# weight = torch.randn(patch_depth,model_dim)

kernle = weight.transpose(0,1).reshape((-1,ic,patch_size,patch_size)) #kernel要满足 oc*ic*kh*kw
patch_embedding_conv = image2emb_conv(image,kernle,patch_size)
print(patch_embedding_conv.shape,patch_embedding_conv)

torch.Size([1, 4, 8]) tensor([[[ 14.4183,   4.0192,   4.9964,  -9.4050,   7.4996,  -1.2957,  10.1465,
           -7.6784],
         [-11.7635,   7.4509,   0.2277, -16.4446,  -4.2662,   0.7789,   7.5461,
           -3.1969],
         [ -6.7492,   4.5716,   7.7062,  -4.8321,   3.8501,  -6.5702, -14.2348,
            5.2437],
         [  0.8196,   9.2905,  -2.3660,   4.6366,  -5.8006,   4.6970,   0.5125,
           -4.0809]]])


In [5]:
#-----------------------------------------------------------------------------------------
# 上面完成了图片的embedding

In [6]:

# 第二种方式，卷积的方式：
def image2emb_conv(image,kernel,stride):
    conv_output = F.conv2d(image,kernel,stride=stride)
    bs,oc,ow,oh = conv_output.shape

    # 还是那个事，transfomer的输入形状是 NLC，L在图像中就是 W*H ，也就是序列长度；
    patch_embedding = conv_output.reshape((bs,oc,ow*oh)).transpose(-1,-2)
    return patch_embedding
bs,ic,image_h,image_w = 1,3,8,8
patch_size = 4
model_dim = 8 #目标要输出的通道数目；
patch_depth = patch_size*patch_size*ic
max_num_token = 16 #假设最大只能输入16个切割好的patch图像
num_classes = 10 # 假设有十个分类别
# image = torch.randn(bs,ic,image_h,image_w)
# weight = torch.randn(patch_depth,model_dim)

kernle = weight.transpose(0,1).reshape((-1,ic,patch_size,patch_size)) #kernel要满足 oc*ic*kh*kw
patch_embedding_conv = image2emb_conv(image,kernle,patch_size)
print(patch_embedding_conv.shape,patch_embedding_conv)

torch.Size([1, 4, 8]) tensor([[[ 14.4183,   4.0192,   4.9964,  -9.4050,   7.4996,  -1.2957,  10.1465,
           -7.6784],
         [-11.7635,   7.4509,   0.2277, -16.4446,  -4.2662,   0.7789,   7.5461,
           -3.1969],
         [ -6.7492,   4.5716,   7.7062,  -4.8321,   3.8501,  -6.5702, -14.2348,
            5.2437],
         [  0.8196,   9.2905,  -2.3660,   4.6366,  -5.8006,   4.6970,   0.5125,
           -4.0809]]])


In [8]:
#-----------------------------------------------------------------------------------------
# cls_token 部分：
cls_token_embedding = torch.randn(bs,1,model_dim,requires_grad=True)
token_embedding = torch.cat([cls_token_embedding,patch_embedding_conv],dim=1)

# 位置编码：
position_embedding_table = torch.randn(max_num_token,model_dim,requires_grad=True)
segment_length = token_embedding.shape[1]

# 将位置embedding复制bs份
position_embedding = torch.tile(position_embedding_table[:segment_length],[token_embedding.shape[0],1,1])
token_embedding += position_embedding

#调用包
encoder_layer = nn.TransformerEncoderLayer(d_model=model_dim,nhead=8)
transformer_encoder = nn.TransformerEncoder(encoder_layer,num_layers=6)
encoder_output = transformer_encoder(token_embedding)

# 分类
cls_token_output = encoder_output[:,0,:] #只取输出的第一个 序列，这个就是cls token

linear_layer = nn.Linear(model_dim,num_classes)

logits = linear_layer(cls_token_output)

loss_fn = nn.CrossEntropyLoss()

label = torch.randint(10,(bs,))

loss = loss_fn(logits,label)
print(loss)


tensor(2.1155, grad_fn=<NllLossBackward0>)
